In [1]:
#详细注释在Yolo2_learning.ipynb中import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import cv2
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Lambda, Conv2D
from tensorflow.keras.models import load_model, Model
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body,yolo_eval
import yolo_utils
import time


yolo_model = load_model("model_data/yolo.h5")
class_names = yolo_utils.read_classes("model_data/coco_classes.txt")
anchors = yolo_utils.read_anchors("model_data/yolo_anchors.txt")
num_classes = len(class_names)

In [3]:
cam = cv2.VideoCapture(0 + cv2.CAP_DSHOW)
if not cam.read()[0]:
    cam = cv2.VideoCapture(1 + cv2.CAP_DSHOW)

cam.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

image_shape = (480., 640.)
model_image_size = (608, 608)
while(True):
    ret, frame = cam.read()
    image = PIL.Image.fromarray(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
    resized_image = image.resize(tuple(reversed(model_image_size)), PIL.Image.BICUBIC)
    image_data = np.array(resized_image, dtype='float32')
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)  # Add batch dimension.
    yolo_outputs = yolo_model(image_data)
    yolo_outputs = yolo_head(yolo_outputs, anchors, num_classes)
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs
    boxes = yolo_boxes_to_corners(box_xy, box_wh)
    box_scores = box_confidence * box_class_probs
    box_classes = K.argmax(box_scores, axis=-1)
    box_class_scores = K.max(box_scores, axis=-1)
    prediction_mask = box_class_scores >= 0.6
    boxes = tf.boolean_mask(boxes, prediction_mask)
    scores = tf.boolean_mask(box_class_scores, prediction_mask)
    classes = tf.boolean_mask(box_classes, prediction_mask)
    height = image_shape[0]
    width = image_shape[1]
    image_dims = K.stack([height, width, height, width])
    image_dims = K.reshape(image_dims, [1, 4])
    boxes = boxes * image_dims
    nms_index = tf.image.non_max_suppression(boxes, scores, 10, iou_threshold=0.5)
    #非极大抑制算法进一步过滤box
    boxes = K.gather(boxes, nms_index)
    scores = K.gather(scores, nms_index)
    classes = K.gather(classes, nms_index)
    colors = yolo_utils.generate_colors(class_names)

    yolo_utils.draw_boxes(image, scores, boxes, classes, class_names, colors)

    image = cv2.cvtColor(np.asarray(image),cv2.COLOR_RGB2BGR)
    cv2.imshow('Cap',image)
    key = cv2.waitKey(1)
    if key == ord("a"):
        break
cam.release()
cv2.destroyAllWindows()